In [1]:
import logging
import os
import sys
import sumolib
import traci

import pandas as pd
import numpy as np
from scipy.sparse import bsr_matrix
from scipy.optimize import lsq_linear
from typing import Dict, Optional, NoReturn, Union, Tuple
import random

from collections import Counter
import shutil
from shapely.geometry import LineString, Polygon


sumolib.os.environ["SUMO_HOME"]="/home/kaveh/build/sumo"

if 'SUMO_HOME' in sumolib.os.environ:
    tools = sumolib.os.path.join(sumolib.os.environ['SUMO_HOME'], 'tools')
    sumolib.sys.path.append(tools)
else:   
    sumolib.sys.exit("please declare environment variable 'SUMO_HOME'")

#sumoBinary = "/home/kaveh/notebook/jupyterenv/bin/sumo"
sumoBinary = "/home/kaveh/virenv/bin/sumo"




In [2]:
from collections import Counter


def vehroute2sensor(vehroute, sensor_set, interval, interval_size):
  
    vehroute["vehicle_arrival"] = vehroute["vehicle_arrival"].fillna(-1)
    vehroute_remined = vehroute[vehroute["vehicle_depart"]!=-1]
    
    sensor_count_list = list()
    
    for index,row in vehroute_remined.iterrows():
        edgelist = row.route_edges.split(" ")
        timelist = [int(i.split(".")[0]) for i in row.route_exitTimes.split(" ")]
        for i in range(len(edgelist)):
            if edgelist[i] in sensor_set:
                if (timelist[i]!= -1) and (timelist[i] >= (interval*interval_size) ):
                    sensor_count_list.append(edgelist[i])
    sensor_count = dict(Counter(sensor_count_list))  
    for s in sensor_set:
        if s not in sensor_count.keys():
            sensor_count[s] = 0
    return sensor_count


def sensor_result_comparing(output_dir, interval, iteration, sample_iteration,  interval_size=3600, no_sensor=45):
    sumolib.subprocess.call(["python3", tools+"/xml/xml2csv.py", output_dir+str(interval)+"/"+str(iteration)+"_"+str(sample_iteration)+"_vehroute_output.xml"])
    vehroute = pd.read_csv(output_dir+str(interval)+"/"+str(iteration)+"_"+str(sample_iteration)+"_vehroute_output.csv", sep=";")
    sensor = pd.read_csv(output_dir + "real_data.csv")
    #print(sensor.head())

    compare = sensor[sensor["interval"]==interval].drop(columns=["interval"])
    #print(compare.head())

    sensor_set = set(sensor["edge"])
    veh_sensor = vehroute2sensor(vehroute, sensor_set, interval, interval_size)
    compare["sim_count"] = compare["edge"].apply(lambda x: veh_sensor[x])#.sort_values("id")
    compare = compare.sort_values("id")
    data = np.load(output_dir+str(interval)+"/"+str(iteration)+"_calib.npz")
    b = data["b"]
    A = data["A"]
    x = data["x"]
    compare["calib"] = list(np.dot(A,x))
    compare["calib_error"] = list((np.dot(A,x)-b[0:no_sensor]))
    compare["b"] = list(b[0:no_sensor])

    return compare

In [3]:
def norm(A):
    return np.sqrt(np.sum(np.square(A)))

def error(A,B, type_error=0):
    if type_error==0:
        return norm(A-B)/norm(A)
    elif type_error==1:
        return norm(A-B)/np.sqrt(len(A))
    else:
        temp = sum(A)/len(A)
        return (norm(A-B)/np.sqrt(len(A)))/temp
    return np.NaN

In [4]:
import plotly.express as px
import plotly.graph_objects as go

def draw_check_error(A, col1="count" , col2="sim_count", interval=0):
    q1 = A[col1].min()
    q2 = A[col2].min()
    q_min = min(q1,q2)-10

    q1 = A[col1].max()
    q2 = A[col2].max()
    q_max = max(q1,q2)+10

    fig = go.Figure()

    fig = px.scatter(A, x=col1, y=col2)#, trendline="ols")#, name="error line")
    fig.add_shape(
        type="line",
        x0=q_min,
        y0=q_min,
        x1=q_max,
        y1=q_max,
        line=dict(width=1, dash="dash"),
    )
    #fig.update_yaxes(
    #    scaleanchor = "x",
    #    scaleratio = 1,
    #  )
    fig.update_xaxes(range=[q_min, q_max], fixedrange=True)
    fig.update_yaxes(range=[q_min, q_max], fixedrange=True)
    fig.update_layout(#title='Average High and Low Temperatures in New York',
                       xaxis_title='Ground truth count',
                       yaxis_title='Simulated count')
    fig.update_layout(showlegend=True, legend_title_text='Trend')
    fig.update_layout(legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    ))

    fig.write_image("images/fig_"+str(interval)+".png")
    fig.show()

In [5]:
output_dir = "output-tartu_2022-09-06_v2/"
report = pd.read_csv(output_dir + "report.csv")
print(len(report))
best_iteration_error_result = report.groupby("interval").apply(lambda table:table.iloc[table["sensor_error_simulation"].argmin()])
best_iteration_error_result

1620


,interval,iteration,vehicle_nummber_calib,sample_number,vehicle_number_sampling,sensor_error_calibration,sensor_error_simulation,sensor_error_sim2calib,running,waiting,speed,fixed_point_error,teleport
interval,,,,,,,,,,,,,
0,0.0,3.0,1064.0,6.0,1075.0,5.072164e-09,0.082624,0.082624,145.0,0.0,46.692,0.063616,0.0
1,1.0,0.0,671.0,0.0,693.0,4.375399e-10,0.071910,0.071910,79.0,0.0,46.584,0.119776,0.0
2,2.0,6.0,569.0,4.0,581.0,1.947997e-09,0.099455,0.099455,71.0,0.0,46.800,0.100563,0.0
3,3.0,1.0,656.0,8.0,662.0,2.453923e-09,0.120123,0.120123,74.0,0.0,46.764,0.070964,0.0
4,4.0,2.0,1089.0,2.0,1063.0,1.151667e-09,0.094577,0.094577,142.0,0.0,47.556,0.124618,0.0
5,5.0,0.0,2281.0,11.0,2272.0,7.763197e-10,0.074195,0.074195,258.0,0.0,46.836,0.127376,0.0
6,6.0,0.0,7120.0,12.0,7239.0,1.350878e-09,0.034469,0.034469,889.0,0.0,44.820,0.157646,2.0
7,7.0,1.0,12950.0,2.0,12962.0,3.465537e-02,0.096855,0.091053,2611.0,28.0,37.548,0.311664,25.0
8,8.0,7.0,14705.0,14.0,14729.0,1.692278e-02,0.127448,0.126524,3388.0,29.0,33.048,0.264885,163.0


In [6]:
A = sensor_result_comparing(output_dir,12,0,2, interval_size=3600, no_sensor=33)
print(A.head())
print(error(A["count"], A["sim_count"], 0))

              edge  id       count  sim_count       calib  calib_error  \
12   -1033719114#3   0   66.333333         66   53.332436    -0.000897   
36    -215453906#2   1  390.666667        385  355.456422    -0.210244   
60    -223572253#3   2  282.666667        289  269.473828    -0.192838   
84   -223600267#12   3  287.333333        302  280.082470    -0.250863   
108     -294173664   4  377.333333        386  377.201427    -0.131906   

              b  
12    53.333333  
36   355.666667  
60   269.666667  
84   280.333333  
108  377.333333  
0.05235419209959357


In [9]:
import os

if not os.path.exists("images"):
    os.mkdir("images")
    
for i in range(24):
    interval  = int(best_iteration_error_result.iloc[i]["interval"])
    iteration = int(best_iteration_error_result.iloc[i]["iteration"])
    sample_number = int(best_iteration_error_result.iloc[i]["sample_number"])
    A = sensor_result_comparing(output_dir,interval,iteration,sample_number, interval_size=3600, no_sensor=33)
    draw_check_error(A, interval=interval)

In [12]:
best_iteration_error_result["waiting"].shift(1).fillna(0)

interval
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8      28.0
9      29.0
10      4.0
11      1.0
12      0.0
13      1.0
14     21.0
15     98.0
16    169.0
17    409.0
18    295.0
19    316.0
20    193.0
21     20.0
22      0.0
23      0.0
Name: waiting, dtype: float64

In [22]:
best_iteration_error_result["trips"] = best_iteration_error_result["vehicle_number_sampling"] - \
                                       best_iteration_error_result["waiting"] + \
                                       best_iteration_error_result["waiting"].shift(1).fillna(0)

In [26]:
best_iteration_error_result["trips"] = best_iteration_error_result["trips"].astype(int)

In [27]:
best_iteration_error_result

,interval,iteration,vehicle_nummber_calib,sample_number,vehicle_number_sampling,sensor_error_calibration,sensor_error_simulation,sensor_error_sim2calib,running,waiting,speed,fixed_point_error,teleport,trips,RError,RMSE,NRMSE
interval,,,,,,,,,,,,,,,,,
0,0.0,3.0,1064.0,6.0,1075.0,5.072164e-09,0.082624,0.082624,145.0,0.0,46.692,0.063616,0.0,1075,0.082624,2.478697,0.140424
1,1.0,0.0,671.0,0.0,693.0,4.375399e-10,0.071910,0.071910,79.0,0.0,46.584,0.119776,0.0,693,0.071910,1.750421,0.117247
2,2.0,6.0,569.0,4.0,581.0,1.947997e-09,0.099455,0.099455,71.0,0.0,46.800,0.100563,0.0,581,0.099455,1.762879,0.160115
3,3.0,1.0,656.0,8.0,662.0,2.453923e-09,0.120123,0.120123,74.0,0.0,46.764,0.070964,0.0,662,0.120123,2.053165,0.174176
4,4.0,2.0,1089.0,2.0,1063.0,1.151667e-09,0.094577,0.094577,142.0,0.0,47.556,0.124618,0.0,1063,0.094577,2.962163,0.126131
5,5.0,0.0,2281.0,11.0,2272.0,7.763197e-10,0.074195,0.074195,258.0,0.0,46.836,0.127376,0.0,2272,0.074195,4.546061,0.090247
6,6.0,0.0,7120.0,12.0,7239.0,1.350878e-09,0.034469,0.034469,889.0,0.0,44.820,0.157646,2.0,7239,0.034469,6.059470,0.040814
7,7.0,1.0,12950.0,2.0,12962.0,3.465537e-02,0.096855,0.091053,2611.0,28.0,37.548,0.311664,25.0,12934,0.096855,30.790647,0.114644
8,8.0,7.0,14705.0,14.0,14729.0,1.692278e-02,0.127448,0.126524,3388.0,29.0,33.048,0.264885,163.0,14728,0.127448,48.719910,0.148545


In [28]:
RMSE = list()
NRMSE = list()
RError = list()
for i in range(24):
    interval  = int(best_iteration_error_result.iloc[i]["interval"])
    iteration = int(best_iteration_error_result.iloc[i]["iteration"])
    sample_number = int(best_iteration_error_result.iloc[i]["sample_number"])
    A = sensor_result_comparing(output_dir,interval,iteration,sample_number, interval_size=3600, no_sensor=33)
    RError.append(error(A["count"], A["sim_count"], 0))
    RMSE.append(error(A["count"], A["sim_count"], 1))
    NRMSE.append(error(A["count"], A["sim_count"], 2))

In [29]:
best_iteration_error_result["RError"] = RError
best_iteration_error_result["RMSE"] = RMSE
best_iteration_error_result["NRMSE"] = NRMSE

In [30]:
best_iteration_error_result

,interval,iteration,vehicle_nummber_calib,sample_number,vehicle_number_sampling,sensor_error_calibration,sensor_error_simulation,sensor_error_sim2calib,running,waiting,speed,fixed_point_error,teleport,trips,RError,RMSE,NRMSE
interval,,,,,,,,,,,,,,,,,
0,0.0,3.0,1064.0,6.0,1075.0,5.072164e-09,0.082624,0.082624,145.0,0.0,46.692,0.063616,0.0,1075,0.082624,2.478697,0.140424
1,1.0,0.0,671.0,0.0,693.0,4.375399e-10,0.071910,0.071910,79.0,0.0,46.584,0.119776,0.0,693,0.071910,1.750421,0.117247
2,2.0,6.0,569.0,4.0,581.0,1.947997e-09,0.099455,0.099455,71.0,0.0,46.800,0.100563,0.0,581,0.099455,1.762879,0.160115
3,3.0,1.0,656.0,8.0,662.0,2.453923e-09,0.120123,0.120123,74.0,0.0,46.764,0.070964,0.0,662,0.120123,2.053165,0.174176
4,4.0,2.0,1089.0,2.0,1063.0,1.151667e-09,0.094577,0.094577,142.0,0.0,47.556,0.124618,0.0,1063,0.094577,2.962163,0.126131
5,5.0,0.0,2281.0,11.0,2272.0,7.763197e-10,0.074195,0.074195,258.0,0.0,46.836,0.127376,0.0,2272,0.074195,4.546061,0.090247
6,6.0,0.0,7120.0,12.0,7239.0,1.350878e-09,0.034469,0.034469,889.0,0.0,44.820,0.157646,2.0,7239,0.034469,6.059470,0.040814
7,7.0,1.0,12950.0,2.0,12962.0,3.465537e-02,0.096855,0.091053,2611.0,28.0,37.548,0.311664,25.0,12934,0.096855,30.790647,0.114644
8,8.0,7.0,14705.0,14.0,14729.0,1.692278e-02,0.127448,0.126524,3388.0,29.0,33.048,0.264885,163.0,14728,0.127448,48.719910,0.148545


In [33]:
best_iteration_error_result[["trips","speed","RError","RMSE","NRMSE"]].to_csv("tartu_error.csv")

In [62]:
import plotly.express as px
fig = px.bar(best_iteration_error_result, x='interval', y='speed')
fig.update_layout(#title='Average High and Low Temperatures in New York',
                       xaxis_title='Hour',
                       yaxis_title='Average speed (km/h)',
                       font_size = 23)
fig.show()

In [61]:
import plotly.express as px
fig = px.bar(best_iteration_error_result, x='interval', y='trips')
fig.update_layout(#title='Average High and Low Temperatures in New York',
                       xaxis_title='Hour',
                       yaxis_title='Number of trips',
                       font_size = 23)
fig.show()

In [ ]:
#########################################################
####### ####     OD Matrix       ########################
#########################################################

In [45]:
import json
  
# reading the data from the file
with open('output-tartu_2022-09-06/option.txt') as f:
    data = f.read()
option = json.loads(data)

In [42]:
def vehroute2od(output_dir, interval, iteration, sample_iteration):
    sumolib.subprocess.call(["python3", tools+"/xml/xml2csv.py", output_dir+str(interval)+"/"+str(iteration)+"_"+str(sample_iteration)+"_vehroute_output.xml"])
    vehroute = pd.read_csv(output_dir+str(interval)+"/"+str(iteration)+"_"+str(sample_iteration)+"_vehroute_output.csv", sep=";")
    vehroute["vehicle_arrival"] = vehroute["vehicle_arrival"].fillna(-1)
    vehroute = vehroute[vehroute["vehicle_depart"]!=-1]
    vehroute["from"] = vehroute["route_edges"].apply(lambda x: x.split(" ")[0])
    vehroute["to"] = vehroute["route_edges"].apply(lambda x: x.split(" ")[-1])
    od = vehroute.groupby(["from","to"]).apply(len).reset_index().rename(columns={0:"count"})
    od["interval"] = interval
    return od

In [44]:
i = 10
interval  = int(best_iteration_error_result.iloc[i]["interval"])
iteration = int(best_iteration_error_result.iloc[i]["iteration"])
sample_number = int(best_iteration_error_result.iloc[i]["sample_number"])
od_new = vehroute2od(output_dir, interval, iteration, sample_number)
od_new

,from,to,count,interval
0,-1033622463#1,-103916811#7,1,10
1,-1033622463#1,-126777051#0,1,10
2,-1033622463#1,-126777056#5,1,10
3,-1033622463#1,-126777064#1,2,10
4,-1033622463#1,-223099572#4,1,10
...,...,...,...,...
7643,998025021#0,30067639#5,1,10
7644,998025021#0,653798790,1,10
7645,998025021#0,675849066#0,1,10
7646,998025021#0,827439974#5,1,10


In [46]:
net = sumolib.net.readNet(option["net_file"] , withInternal=False)


In [52]:
od_new["from_node"] = od_new["from"].apply(lambda x: net.getEdge(x).getFromNode().getID())
od_new["to_node"] = od_new["to"].apply(lambda x: net.getEdge(x).getToNode().getID())
od_new = od_new.groupby(["from_node","to_node"])["count"].sum().reset_index()
od_new.rename(columns={"from_node":"from", "to_node":"to"}, inplace=True)

In [53]:
od_new

,from,to,count
0,10981451,1157945201,3
1,10981451,1199475134,1
2,10981451,1404484701,2
3,10981451,1404484723,1
4,10981451,1870777763,3
...,...,...,...
6725,cluster_330045496_330045504_5200254966_5200254968,578673912,1
6726,cluster_330045496_330045504_5200254966_5200254968,6138125018,1
6727,cluster_330045496_330045504_5200254966_5200254968,6793225631,2
6728,cluster_330045496_330045504_5200254966_5200254968,7131789198,1


In [54]:
od = pd.read_csv("tartu_2022-09-06/init_dod.csv")
od

,trip_id,from_node,to_node,weight_trip
0,0,10981451,11231981,0.000049
1,1,10981451,1157945201,0.000034
2,2,10981451,1199475134,0.000036
3,3,10981451,1404484701,0.000036
4,4,10981451,1404484711,0.000036
...,...,...,...,...
28815,28815,cluster_330045496_330045504_5200254966_5200254968,976528927,0.000082
28816,28816,cluster_330045496_330045504_5200254966_5200254968,cluster_291381188_330041233,0.000061
28817,28817,cluster_330045496_330045504_5200254966_5200254968,cluster_3234018651_330293737,0.000075
28818,28818,cluster_330045496_330045504_5200254966_5200254968,cluster_330041132_330267660_418091425,0.000032
